# Titanic surviver classification

In [52]:
import csv
import tensorflow as tf
import numpy as np
from tflearn.data_utils import shuffle

In [53]:
train_data = []
with open('Data/train.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        train_data.append(row)
train_data = np.array(train_data)

In [54]:
for col,num in zip(train_data[0], range(len(train_data[0]))):
    print('{}: {}'.format(num,col))
    

0: PassengerId
1: Survived
2: Pclass
3: Name
4: Sex
5: Age
6: SibSp
7: Parch
8: Ticket
9: Fare
10: Cabin
11: Embarked


In [55]:
labels = train_data[1:,1]
labels = np.float64(labels)


In [69]:
def prep_data(data):
    sexind = int(*np.where(data[0] == 'Sex'))
    ageind = int(*np.where(data[0] == 'Age'))
    trimmed_x = data[1:,(sexind,ageind)]
    trimmed_x[trimmed_x == ''] = '0'
    trimmed_x[trimmed_x == 'male'] = '0'
    trimmed_x[trimmed_x == 'female'] = '1'

    trimmed_x = np.float64(trimmed_x)

    ages = trimmed_x[:,1]
    ages = ages[ages != 0]
    mean = np.mean(ages)
    trimmed_x[:,1][trimmed_x[:,1] == 0] = mean

    norm_x = np.copy(trimmed_x)
    mean_age = np.mean(norm_x[:,1])
    for i in range(len(norm_x[:,1])):
        norm_x[:,1][i] = (norm_x[:,1][i] / (mean_age)) - 1
    return norm_x

In [70]:
train_x = prep_data(train_data)

4 5


# model

In [71]:
y = tf.placeholder(tf.float64,[None,1])
x = tf.placeholder(tf.float64, [None,2])
learn_rate = 0.0001
batch_size = 16
epochs = 5

In [72]:
l1 = tf.layers.dense(x, 500, activation=tf.nn.relu)

l2= tf.layers.dense(l1, 50, activation=tf.nn.relu)

output = tf.layers.dense(l2,1, activation=None)

In [73]:
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=output, name='entropy'), name='loss')
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)

predictions = tf.cast(tf.greater(tf.sigmoid(output),.5),tf.float64)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, y), tf.float32))
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [74]:
# Start training
with tf.Session() as sess:
    display_step = 20
    # Run the initializer
    sess.run(init)
    num_batches = int(len(train_x)/batch_size)
    for epoch in range(epochs):
        total_acc = 0
        total_loss = 0
        epoch_x, epoch_y = shuffle(train_x,labels)
        for batch in range(num_batches):
            batch_x = epoch_x[batch * batch_size:(batch*batch_size) + batch_size]
            batch_y = epoch_y[batch*batch_size:(batch*batch_size) + batch_size]
            batch_y =np.reshape(batch_y, (batch_y.shape[0], 1))
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={x: batch_x, y: batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x,
                                                                     y: batch_y})
            total_acc += acc
            total_loss += loss
            if batch % display_step == 0 and batch != 0:
                # Calculate batch loss and accuracy
                
                print("epoch {}, ".format(epoch) + "batch " + str(batch) + ", Loss= " + "{}".format(total_loss/display_step) + ", Accuracy= " + "{}".format(total_acc/display_step))
                total_acc = 0
                total_loss = 0
    saver.save(sess, './80')
    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    #print("Testing Accuracy:", \
      #  sess.run(accuracy, feed_dict={x: mnist.test.images,
       #                               y: mnist.test.labels}))

epoch 0, batch 20, Loss= 0.6760880700556859, Accuracy= 0.796875
epoch 0, batch 40, Loss= 0.5707455682517499, Accuracy= 0.790625
epoch 1, batch 20, Loss= 0.5560277247486409, Accuracy= 0.821875
epoch 1, batch 40, Loss= 0.5072581579192347, Accuracy= 0.79375
epoch 2, batch 20, Loss= 0.5302336417233307, Accuracy= 0.825
epoch 2, batch 40, Loss= 0.5463586674933446, Accuracy= 0.7625
epoch 3, batch 20, Loss= 0.5312858558503499, Accuracy= 0.825
epoch 3, batch 40, Loss= 0.5211701934284109, Accuracy= 0.775
epoch 4, batch 20, Loss= 0.5078751320796902, Accuracy= 0.853125
epoch 4, batch 40, Loss= 0.5132736612816504, Accuracy= 0.775
Optimization Finished!


In [75]:
test_data = []
with open('Data/test.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        test_data.append(row)
test_data = np.array(test_data)

In [79]:
IDs = test_data[1:,0]
test_x = prep_data(test_data)

3 4


In [85]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, "./80")
    pred = sess.run(predictions, feed_dict={x:test_x})
    

INFO:tensorflow:Restoring parameters from ./80


In [92]:
len(pred)==len(IDs)

True

In [108]:
with open("results.csv", 'w') as file:
    feilds = ('PassengerId','Survived')
    writer = csv.DictWriter(file,feilds, delimiter=',', lineterminator = '\n' )
    writer.writeheader()
    for ID, prediction in zip(IDs,pred):
        writer.writerow({'PassengerId':str(ID), 'Survived': str(int(prediction))})

## now with a random forest classifier

In [112]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def prep_data(data):
    sexind = int(*np.where(data[0] == 'Sex'))
    ageind = int(*np.where(data[0] == 'Age'))
    trimmed_x = data[1:,(sexind,ageind)]
    trimmed_x[trimmed_x == ''] = '0'
    trimmed_x[trimmed_x == 'male'] = '0'
    trimmed_x[trimmed_x == 'female'] = '1'

    trimmed_x = np.float64(trimmed_x)

    ages = trimmed_x[:,1]
    ages = ages[ages != 0]
    mean = np.mean(ages)
    trimmed_x[:,1][trimmed_x[:,1] == 0] = mean

    norm_x = np.copy(trimmed_x)
    mean_age = np.mean(norm_x[:,1])
    for i in range(len(norm_x[:,1])):
        norm_x[:,1][i] = (norm_x[:,1][i] / (mean_age)) - 1
    return norm_x

In [128]:

model = RandomForestClassifier(n_estimators=100)
model.fit(train_x, labels)
score = model.score(train_x, labels)
pred = model.predict(test_x)
score

0.81930415263748602

In [129]:
with open("results_RFC.csv", 'w') as file:
    feilds = ('PassengerId','Survived')
    writer = csv.DictWriter(file,feilds, delimiter=',', lineterminator = '\n' )
    writer.writeheader()
    for ID, prediction in zip(IDs,pred):
        writer.writerow({'PassengerId':str(ID), 'Survived': str(int(prediction))})